# SK-68D16

Updating the dataset: E511220100000

* All channels have drop-outs; none are photometric.  
* Scale LiF 2A and 1B to match STIS spectrum.  
* Boot-strap to shorter wavelengths.  
* Use normal selection of channels.

COMMENT File updated 11 May 2022<br>
COMMENT This NVO file has been reconstructed from the corresponding ALL file.<br>
COMMENT All segments have been scaled to match a STIS spectrum of this star. 

In [ ]:
%matplotlib inline
import matplotlib

import numpy as np

from astropy import units as u
from astropy.convolution import convolve, Box1DKernel
from astropy.io import fits
from astropy.visualization import quantity_support
from matplotlib import pyplot as plt

from specutils import Spectrum1D
from specutils.spectra import SpectralRegion
from specutils.manipulation import box_smooth, extract_region

# Specify plot parameters.
quantity_support()  # to put units on the axes below 
matplotlib.rcParams['figure.figsize'] = [15, 5]

# Wavelength per channel (pixel) is always 0.013 A.
WPC = 0.013

# Normalize all spectra, so fluxes are of order unity.
norm = 1E12

In [ ]:
# Function to convert FITS array into spectrum object.

def make_spectrum (specdata):
    lamb = specdata['wave'] * u.AA 
    flux = specdata['flux'] * norm * u.Unit('erg cm-2 s-1 AA-1') 
    return Spectrum1D(spectral_axis=lamb, flux=flux)

In [ ]:
# Function to compute offset in pixels between two spectra

def compute_shift(specdata, refdata, wmin, wmax):
    wave_spec  = specdata['wave']
    flux_spec  = specdata['flux']
    error_spec = specdata['error']
    wave_ref   =  refdata['wave']
    flux_ref   =  refdata['flux']
    
    # Compute scale factor between data and reference spectra.
    g = np.where((wave_spec > wmin) & (wave_spec < wmax))
    h = np.where((wave_ref  > wmin) & (wave_ref  < wmax))
    spec_mean = np.mean(flux_spec[g])
    ref_mean  = np.mean(flux_ref[h])
    scale = ref_mean / spec_mean

    # Smooth data and error arrays.
    flux_spec  = convolve(flux_spec,  Box1DKernel(7))
    error_spec = convolve(error_spec, Box1DKernel(7))
    flux_ref   = convolve(flux_ref,   Box1DKernel(7))
    
    # Rescale spectrum to match reference.
    flux_spec *= scale
    error_spec *= scale
    
    # Compute minimum value of chi-squared.
    chisq = np.zeros(20)
    for i in range(20):
        j = i - 10
        chisq[i] = np.sum(((flux_spec[g[0]+j] - flux_ref[h]) / error_spec[g[0]+j])**2)
    
    # We have computed this offset by shifting the flux array, but will 
    # use it by shifting the wavelength array, so must multiply by -1.
    return 10 - np.argmin(chisq)

In [ ]:
# Read header keywords.

filename = 'e511220100000all2ttagfcal.fit'
f = fits.open(filename)
print ('Target:  ', f[0].header['TARGNAME'])
print ('Aperture:', f[0].header['APERTURE'])
print ('Guider:  ', f[0].header['FESCENT'])
print ('CHANL OBSTIME COMBMETH')
for i in range(1,9): print (f[i].header['EXTNAME'], f[i].header['OBSTIME'], f[i].header['COMBMETH'])
    
# LiF 1 is the target channel.

In [ ]:
# Read all eight spectra from FUSE *all* file.

lif1a_data = f['1alif'].data 
lif1b_data = f['1blif'].data
lif2b_data = f['2blif'].data
lif2a_data = f['2alif'].data
sic1a_data = f['1asic'].data
sic1b_data = f['1bsic'].data
sic2b_data = f['2bsic'].data
sic2a_data = f['2asic'].data
f.close() 

In [ ]:
# Convert FITS arrays into spectrum objects.

lif1a = make_spectrum (lif1a_data)
lif1b = make_spectrum (lif1b_data)
lif2a = make_spectrum (lif2a_data)
lif2b = make_spectrum (lif2b_data)
sic1a = make_spectrum (sic1a_data)
sic1b = make_spectrum (sic1b_data)
sic2a = make_spectrum (sic2a_data)
sic2b = make_spectrum (sic2b_data)

In [ ]:
# Smooth the spectral arrays.

from specutils.manipulation import (box_smooth)

lif1a_bsmooth = box_smooth(lif1a, width=15)
lif1b_bsmooth = box_smooth(lif1b, width=15)
lif2a_bsmooth = box_smooth(lif2a, width=15)
lif2b_bsmooth = box_smooth(lif2b, width=15)
sic1a_bsmooth = box_smooth(sic1a, width=15)
sic1b_bsmooth = box_smooth(sic1b, width=15)
sic2a_bsmooth = box_smooth(sic2a, width=15)
sic2b_bsmooth = box_smooth(sic2b, width=15)

In [ ]:
# Plot the smoothed spectra.  

f, ax = plt.subplots()  
ax.step(lif1a_bsmooth.spectral_axis, lif1a_bsmooth.flux, label='LiF1A') 
ax.step(lif2b_bsmooth.spectral_axis, lif2b_bsmooth.flux, label='LiF2B') 
ax.step(sic1a_bsmooth.spectral_axis, sic1a_bsmooth.flux, label='SiC1A') 
ax.step(sic2b_bsmooth.spectral_axis, sic2b_bsmooth.flux, label='SiC2B') 
ax.legend()
ax.set_xlim([1035, 1085])
ax.set_ylim([0,1.2])

# The count-rate plots show drop-outs for both of the LiF channels,
# so the actual flux may be much higher.

In [ ]:
# Compare with a STIS spectrum of the same star.

filename = 'oe9s4s020_x1d.fits'
stis = Spectrum1D.read(filename, format="HST/STIS")
stis_bsmooth = box_smooth(stis, width=15)

f, ax = plt.subplots()
ax.plot(stis_bsmooth.spectral_axis, norm * stis_bsmooth.flux, label='STIS')  
ax.step(lif2a_bsmooth.spectral_axis, lif2a_bsmooth.flux, label='LiF 2A') 
ax.plot(lif1b_bsmooth.spectral_axis, lif1b_bsmooth.flux, label='LiF 1B') 
ax.legend()
ax.set_xlim([1140,1200])
ax.set_ylim([0,1.5])

print ('Target:  ', fits.getval(filename, 'TARGNAME'))
print ('Aperture:', fits.getval(filename, 'APERTURE'))

# STIS used a small aperture, so spectrum is probably not contaminated.

# Let's scale the entire FUSE spectrum to match STIS. We'll start at long wavelengths and work our way down.

In [ ]:
# Select a spectral region.

region = SpectralRegion(1155*u.AA, 1180*u.AA)
sub_lif2a = extract_region(lif2a, region)
sub_lif1b = extract_region(lif1b, region)
sub_stis  = extract_region(stis, region)

# Compute ratio of their fluxes.

mean_stis = norm * sub_stis.mean()
scale_lif2a = mean_stis/sub_lif2a.mean()
print ('Scale LiF 2A by', scale_lif2a)
scale_lif1b = mean_stis/sub_lif1b.mean()
print ('Scale LiF 1B by', scale_lif1b)

In [ ]:
# Scale the spectra.

lif2a *= scale_lif2a
lif1b *= scale_lif1b

lif2a_bsmooth *= scale_lif2a
lif1b_bsmooth *= scale_lif1b

In [ ]:
# Plot the rescaled spectra.

f, ax = plt.subplots()  
ax.plot(stis_bsmooth.spectral_axis, norm * stis_bsmooth.flux, label='STIS')  
ax.step(lif2a_bsmooth.spectral_axis, lif2a_bsmooth.flux, label='LiF 2A') 
ax.plot(lif1b_bsmooth.spectral_axis, lif1b_bsmooth.flux, label='LiF 1B')
ax.legend()
ax.set_xlim([1075, 1250])
ax.set_ylim([0,1.5])

In [ ]:
# Now we bootstrap our way to shorter wavelengths.

f, ax = plt.subplots()  
ax.step(lif1a_bsmooth.spectral_axis, lif1a_bsmooth.flux, label='LiF1A') 
ax.step(lif2b_bsmooth.spectral_axis, lif2b_bsmooth.flux, label='LiF2B') 
ax.step(sic1a_bsmooth.spectral_axis, sic1a_bsmooth.flux, label='SiC1A') 
ax.step(sic2b_bsmooth.spectral_axis, sic2b_bsmooth.flux, label='SiC2B') 
ax.step(lif2a_bsmooth.spectral_axis, lif2a_bsmooth.flux, label='LiF2A') 
ax.legend()
ax.set_xlim([1020, 1120])
ax.set_ylim([0,1.2])

In [ ]:
# Scale SiC 2B to match LiF 2A in the region of overlap.

# Select a broad spectral region.

region = SpectralRegion(1087*u.AA, 1096*u.AA)
sub_lif2a = extract_region(lif2a, region)
sub_sic2b = extract_region(sic2b, region)

# Compute ratio of their fluxes.

mean_lif2a = sub_lif2a.mean()
scale_sic2b = mean_lif2a/sub_sic2b.mean()
print ('Scale SiC 2B by', scale_sic2b)

In [ ]:
# Scale the spectra.

sic2b *= scale_sic2b
sic2b_bsmooth *= scale_sic2b

In [ ]:
# Replot the spectra.

f, ax = plt.subplots()  
ax.step(lif1a_bsmooth.spectral_axis, lif1a_bsmooth.flux, label='LiF1A') 
ax.step(lif2b_bsmooth.spectral_axis, lif2b_bsmooth.flux, label='LiF2B') 
ax.step(sic1a_bsmooth.spectral_axis, sic1a_bsmooth.flux, label='SiC1A')
ax.step(sic2b_bsmooth.spectral_axis, sic2b_bsmooth.flux, label='SiC2B')
ax.legend()
ax.set_ylim([0,1.5])

In [ ]:
# Rescale the LiF 1A, LiF 2B, and SiC 1A channels to match SiC 2B.

# Select a broad spectral region.

region = SpectralRegion(1040*u.AA, 1070*u.AA)
sub_lif1a = extract_region(lif1a, region)
sub_lif2b = extract_region(lif2b, region)
sub_sic1a = extract_region(sic1a, region)
sub_sic2b = extract_region(sic2b, region)

# Compute ratio of their fluxes.

mean_sic2b = sub_sic2b.mean()
scale_lif1a = mean_sic2b/sub_lif1a.mean()
print ('Scale LiF 1A by', scale_lif1a)
scale_lif2b = mean_sic2b/sub_lif2b.mean()
print ('Scale LiF 2B by', scale_lif2b)
scale_sic1a = mean_sic2b/sub_sic1a.mean()
print ('Scale SiC 1A by', scale_sic1a)

In [ ]:
# Scale the smoothed spectra.

lif1a *= scale_lif1a
lif2b *= scale_lif2b
sic1a *= scale_sic1a

lif1a_bsmooth *= scale_lif1a
lif2b_bsmooth *= scale_lif2b
sic1a_bsmooth *= scale_sic1a

In [ ]:
# Replot the scaled spectra.

f, ax = plt.subplots()  
ax.step(lif1a_bsmooth.spectral_axis, lif1a_bsmooth.flux, label='LiF1A') 
ax.step(lif2b_bsmooth.spectral_axis, lif2b_bsmooth.flux, label='LiF2B') 
ax.step(sic1a_bsmooth.spectral_axis, sic1a_bsmooth.flux, label='SiC1A')
ax.step(sic2b_bsmooth.spectral_axis, sic2b_bsmooth.flux, label='SiC2B')
ax.legend()
ax.set_ylim([0,1.5])

# This looks fine.

In [ ]:
# We continue to boot-strap our way to shorter wavelengths.

f, ax = plt.subplots() 
ax.step(lif1a_bsmooth.spectral_axis, lif1a_bsmooth.flux, label='LiF 1A') 
ax.step(sic1b_bsmooth.spectral_axis, sic1b_bsmooth.flux, label='SiC 1B')
ax.step(sic2a_bsmooth.spectral_axis, sic2a_bsmooth.flux, label='SiC 2A')
ax.legend()
ax.set_xlim([920, 1020])
ax.set_ylim([0,1.5])

In [ ]:
# First, we use LiF 1A to rescale SiC 2A.

# Select a spectral region.

region = SpectralRegion(995*u.AA, 1005*u.AA)
sub_lif1a = extract_region(lif1a, region)
sub_sic2a = extract_region(sic2a, region)

# Compute ratio of the flux to LiF 1A.

scale_sic2a = sub_lif1a.mean()/sub_sic2a.mean()
print ('Scale SiC 2A by', scale_sic2a)

# Scale the spectrum.

sic2a *= scale_sic2a
sic2a_bsmooth *= scale_sic2a

In [ ]:
# Then use SiC 2A to rescale SiC 1B.

# Select a spectral region.

region = SpectralRegion(957*u.AA, 970*u.AA)
sub_sic1b = extract_region(sic1b, region)
sub_sic2a = extract_region(sic2a, region)

# Compute ratio of the flux to SiC 2A.

scale_sic1b = sub_sic2a.mean()/sub_sic1b.mean()
print ('Scale SiC 1B by', scale_sic1b)

# Scale the spectrum.

sic1b *= scale_sic1b
sic1b_bsmooth *= scale_sic1b

# Plot the scaled spectra.

f, ax = plt.subplots() 
ax.step(lif1a_bsmooth.spectral_axis, lif1a_bsmooth.flux, label='LiF 1A')
ax.step(sic1b_bsmooth.spectral_axis, sic1b_bsmooth.flux, label='SiC 1B')
ax.step(sic2a_bsmooth.spectral_axis, sic2a_bsmooth.flux, label='SiC 2A')
ax.legend()
ax.set_xlim([920, 1020])
ax.set_ylim([0,1.5])

In [ ]:
# Next we compute the shift (in pixels) relative to LiF 1A.

shift_lif2 = compute_shift(lif2b_data, lif1a_data, 1040, 1070)
shift_sic1 = compute_shift(sic1a_data, lif1a_data, 1040, 1070)
shift_sic2 = compute_shift(sic2b_data, lif1a_data, 1040, 1070)

print ('Shift LiF 2 by ', shift_lif2, ' pixels.')
print ('Shift SiC 1 by ', shift_sic1, ' pixels.')
print ('Shift SiC 2 by ', shift_sic2, ' pixels.')

In [ ]:
# Shift all eight spectra.

shift_lif2 *= WPC
shift_sic1 *= WPC
shift_sic2 *= WPC

lif1a_shift = lif1a
lif1b_shift = lif1b
lif2a_shift = Spectrum1D(spectral_axis=lif2a.spectral_axis + shift_lif2 * u.AA, flux=lif2a.flux)
lif2b_shift = Spectrum1D(spectral_axis=lif2b.spectral_axis + shift_lif2 * u.AA, flux=lif2b.flux)
sic1a_shift = Spectrum1D(spectral_axis=sic1a.spectral_axis + shift_sic1 * u.AA, flux=sic1a.flux)
sic1b_shift = Spectrum1D(spectral_axis=sic1b.spectral_axis + shift_sic1 * u.AA, flux=sic1b.flux)
sic2a_shift = Spectrum1D(spectral_axis=sic2a.spectral_axis + shift_sic2 * u.AA, flux=sic2a.flux)
sic2b_shift = Spectrum1D(spectral_axis=sic2b.spectral_axis + shift_sic2 * u.AA, flux=sic2b.flux)

f, ax = plt.subplots()  
ax.step(lif1a.spectral_axis, lif1a_bsmooth.flux, label='LiF 1A') 
ax.step(lif2b_shift.spectral_axis, lif2b_bsmooth.flux, label='LiF 2B')
ax.step(sic1a_shift.spectral_axis, sic1a_bsmooth.flux, label='SiC 1A') 
ax.step(sic2b_shift.spectral_axis, sic2b_bsmooth.flux, label='SiC 2B')
ax.legend()
ax.set_xlim([1060,1070])
ax.set_ylim([0,1.5])

In [ ]:
# Reassemble the NVO file.

filename = 'e511220100000nvo2ttagfcal.fit'
f = fits.open(filename)
hdr = f[0].header
data = f[1].data 

# Scale SiC 1B
sic1b_wave = sic1b_shift.spectral_axis.value
h = np.where((sic1b_wave > 899.99) & (sic1b_wave < 917.5))
g = np.where((data['wave'] > np.min(sic1b_wave[h]) - 0.01) & (data['wave'] < 917.5))
data['flux'][g] = float(scale_sic1b) * sic1b_data['flux'][h]
data['error'][g] = float(scale_sic1b) * sic1b_data['error'][h]

# Scale SiC 2A
sic2a_wave = sic2a_shift.spectral_axis.value
g = np.where((data['wave'] > 917.5) & (data['wave'] < 998))
h = np.where((sic2a_wave > 917.5) & (sic2a_wave < 998))
data['flux'][g] = float(scale_sic2a) * sic2a_data['flux'][h]
data['error'][g] = float(scale_sic2a) * sic2a_data['error'][h]

# Scale LiF 1A
lif1a_wave = lif1a_shift.spectral_axis.value
g = np.where((data['wave'] > 998) & (data['wave'] < 1082.5))
h = np.where((lif1a_wave > 998) & (lif1a_wave < 1082.5))
data['flux'][g] = float(scale_lif1a) * lif1a_data['flux'][h]
data['error'][g] = float(scale_lif1a) * lif1a_data['error'][h]

# Scale SiC 1A 
sic1a_wave = sic1a_shift.spectral_axis.value
g = np.where((data['wave'] > 1082.5) & (data['wave'] < 1087))
h = np.where((sic1a_wave > 1082.5) & (sic1a_wave < 1087))
data['flux'][g] = float(scale_sic1a) * sic1a_data['flux'][h]
data['error'][g] = float(scale_sic1a) * sic1a_data['error'][h]

# Scale LiF 2A
lif2a_wave = lif2a_shift.spectral_axis.value
g = np.where((data['wave'] > 1087) & (data['wave'] < 1180))
h = np.where((lif2a_wave > 1087) & (lif2a_wave < 1180))
data['flux'][g] = float(scale_lif2a) * lif2a_data['flux'][h]
data['error'][g] = float(scale_lif2a) * lif2a_data['error'][h]

# Scale LiF 1B
lif1b_wave = lif1b_shift.spectral_axis.value
h = np.where((lif1b_wave > 1180) & (lif1b_wave < 1190.01))
g = np.where((data['wave'] > 1180) & (data['wave'] < np.max(lif1b_wave[h]) + 0.01))
data['flux'][g] = float(scale_lif1b) * lif1b_data['flux'][h]
data['error'][g] = float(scale_lif1b) * lif1b_data['error'][h]

hdr['comment'] = ''
hdr['comment'] = 'File updated 11 May 2022'
hdr['comment'] = 'This NVO file has been reconstructed from the corresponding ALL file.'
hdr['comment'] = 'All segments have been scaled to match a STIS spectrum of this star.'

f.writeto('level0_e511220100000nvo2ttagfcal_vo.fit', overwrite=True)
f.close()

In [ ]:
# Compare old and new versions of NVO file.

filename = 'e511220100000nvo2ttagfcal.fit'
f = fits.open(filename)
old = f[1].data 
f.close()

filename = 'level0_e511220100000nvo2ttagfcal_vo.fit'
f = fits.open(filename)
hdr = f[0].header
new = f[1].data 
f.close()

print (hdr['comment'])

f, (ax1, ax2) = plt.subplots(2, 1, sharey=True)  
ax1.step(new['wave'], new['flux'], label='NEW FLUX')
ax1.step(old['wave'], old['flux'], label='OLD FLUX')
ax2.step(new['wave'], new['flux'])
ax2.step(old['wave'], old['flux']) 
ax1.legend()
ax1.set_xlim([900, 1050])
ax2.set_xlim([1050, 1190])
ax1.set_ylim([0,1.5E-12])

In [ ]:
# Let's look more closely at the regions sampled only by the SiC channels.

f, ax = plt.subplots()  
ax.step(new['wave'], new['flux'], label='NEW FLUX')
ax.step(old['wave'], old['flux'], label='OLD FLUX') 
ax.legend()
ax.set_xlim([1075,1100])
ax.set_ylim([0,1.5E-12])

# Seems reasonable.